# Joining the 2020 Census data population estimates to Zillow housing data

In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup
import re
from numpy import random
repo1 = 'C:\\Users\\whetz\\Documents\\UVA MSDS\\Bayes Machine Learning\\Project\\bayes_real_estate_modeling'
pd.set_option("display.max_columns", 100)

# Get population data

https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-totals-metro-and-micro-statistical-areas.html

### Population -  Read in data, and get isolated region and state data

In [36]:
os.chdir(repo1)
df_pop2020 = pd.read_csv('cbsa-est2020-alldata.csv')

# Filter for just MSAs
df_pop2020 = df_pop2020.query("LSAD == 'Metropolitan Statistical Area'").reset_index(drop=True)

# Drop unnecessary columns
df_pop2020 = df_pop2020.drop(columns=['CBSA','MDIV','STCOU', 'LSAD'])

# Get the primary (first listed) state of each area
df_pop2020.insert(loc=1, column='state', value=[i.split(', ')[1].split('-')[0] for i in df_pop2020.NAME])

# Get the primary city
df_pop2020.insert(loc=1, column='city', value=[i.split('-')[0] for i in df_pop2020.NAME])
df_pop2020.city = [df_pop2020.city.loc[i].split(',')[0] for i in range(len(df_pop2020))]

print("Rows:",len(df_pop2020))
df_pop2020.sort_values('POPESTIMATE2020', ascending=False).head(10)
# df_pop2020.head()

Rows: 384


,NAME,city,state,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,NPOPCHG2010,NPOPCHG2011,NPOPCHG2012,NPOPCHG2013,NPOPCHG2014,NPOPCHG2015,NPOPCHG2016,NPOPCHG2017,NPOPCHG2018,NPOPCHG2019,NPOPCHG2020,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,BIRTHS2016,BIRTHS2017,BIRTHS2018,BIRTHS2019,BIRTHS2020,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,DEATHS2016,DEATHS2017,DEATHS2018,DEATHS2019,DEATHS2020,NATURALINC2010,...,NATURALINC2015,NATURALINC2016,NATURALINC2017,NATURALINC2018,NATURALINC2019,NATURALINC2020,INTERNATIONALMIG2010,INTERNATIONALMIG2011,INTERNATIONALMIG2012,INTERNATIONALMIG2013,INTERNATIONALMIG2014,INTERNATIONALMIG2015,INTERNATIONALMIG2016,INTERNATIONALMIG2017,INTERNATIONALMIG2018,INTERNATIONALMIG2019,INTERNATIONALMIG2020,DOMESTICMIG2010,DOMESTICMIG2011,DOMESTICMIG2012,DOMESTICMIG2013,DOMESTICMIG2014,DOMESTICMIG2015,DOMESTICMIG2016,DOMESTICMIG2017,DOMESTICMIG2018,DOMESTICMIG2019,DOMESTICMIG2020,NETMIG2010,NETMIG2011,NETMIG2012,NETMIG2013,NETMIG2014,NETMIG2015,NETMIG2016,NETMIG2017,NETMIG2018,NETMIG2019,NETMIG2020,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019,RESIDUAL2020
249,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,18897109,18896285,18923437,19053124,19150453,19227457,19282138,19322239,19336456,19325115,19289420,19232494,19124359,27152,129687,97329,77004,54681,40101,14217,-11341,-35695,-56926,-108135,61878,247572,243023,242529,240755,242697,241453,236744,235154,229366,226416,31931,135400,131949,138295,134684,138029,136119,140309,140953,148762,163362,29947,...,104668,105334,96435,94201,80604,63054,21868,103617,98508,89001,95069,104228,106460,95474,68752,57282,46173,-24138,-86161,-111777,-115585,-146459,-169071,-197835,-203430,-198717,-194533,-216810,-2270,17456,-13269,-26584,-51390,-64843,-91375,-107956,-129965,-137251,-170637,-525,59,-476,-646,0,276,258,180,69,-279,-552
210,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,CA,12828837,12828904,12838023,12923899,13010104,13092451,13159972,13226215,13259142,13266524,13237112,13182453,13109903,9119,85876,86205,82347,67521,66243,32927,7382,-29412,-54659,-72550,41041,170873,166999,168283,167328,167101,159988,159348,150001,144314,142952,18118,75847,75792,78859,76524,79629,82780,82813,84887,87482,96746,22923,...,87472,77208,76535,65114,56832,46206,7605,40264,33724,41167,46197,54218,48748,39948,25488,11676,9909,-21547,-49359,-36684,-46809,-68352,-74778,-92981,-109079,-120151,-123237,-128803,-13942,-9095,-2960,-5642,-22155,-20560,-44233,-69131,-94663,-111561,-118894,138,-55,-2042,-1435,-1128,-669,-48,-22,137,70,138
68,"Chicago-Naperville-Elgin, IL-IN-WI",Chicago,IL,9461105,9461540,9470661,9500919,9528134,9550227,9560285,9552351,9533732,9513947,9485403,9454282,9406638,9121,30258,27215,22093,10058,-7934,-18619,-19785,-28544,-31121,-47644,32252,123459,121750,119170,119145,120059,117796,113474,111301,107294,106292,16097,66706,66893,68803,67701,71108,70153,72507,73314,74556,81256,16155,...,48951,47643,40967,37987,32738,25036,3911,25473,22153,22233,23468,24536,19728,24054,16672,9890,8482,-10732,-52050,-49504,-50212,-64927,-81686,-86136,-84977,-83378,-73824,-81174,-6821,-26577,-27351,-27979,-41459,-57150,-66408,-60923,-66706,-63934,-72692,-213,82,-291,-295,73,265,146,171,175,75,12
86,"Dallas-Fort Worth-Arlington, TX",Dallas,TX,6366542,6366541,6392097,6510550,6644519,6753804,6889603,7042322,7195475,7337707,7457586,7574390,7694138,25556,118453,133969,109285,135799,152719,153153,142232,119879,116804,119748,23615,95698,95063,95690,97814,99549,100304,99533,98125,97160,97614,8883,37163,37440,39411,40309,41823,42220,44479,45848,47767,52487,14732,...,57726,58084,55054,52277,49393,45127,4092,19843,22130,21802,30420,34612,33416,30276,22088,20938,16894,6353,39956,53322,31209,47714,60142,61552,56872,45394,465

### Population -   Melt the dataframe and get year data so that every row has city, state, year and a census variable

In [37]:
# Melt the dataframe to get one observation per row
id_cols = ['NAME','city','state']
value_cols = [i for i in df_pop2020.columns if i not in id_cols]
df_census_melt = df_pop2020.melt(id_vars=id_cols, value_vars = value_cols, var_name='census_variable', value_name='census_value')

# Get the year
df_census_melt.insert(loc=3, column='year', value = [int(re.findall("[0-9][0-9][0-9][0-9]", i)[0]) for i in df_census_melt.census_variable])

# Remove year from variable
df_census_melt.census_variable = [df_census_melt.census_variable.loc[i].replace(str(df_census_melt.year.loc[i]),'') for i in range(len(df_census_melt))]

# Sort by NAME
df_census_melt = df_census_melt.sort_values(['NAME','census_variable','year'])
df_census_melt.sort_values('census_value', ascending=False)

,NAME,city,state,year,census_variable,census_value
3321,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2016,POPESTIMATE,19336456
3705,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2017,POPESTIMATE,19325115
2937,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2015,POPESTIMATE,19322239
4089,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2018,POPESTIMATE,19289420
2553,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2014,POPESTIMATE,19282138
...,...,...,...,...,...,...
29817,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2019,DOMESTICMIG,-194533
28665,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2016,DOMESTICMIG,-197835
29433,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2018,DOMESTICMIG,-198717
29049,"New York-Newark-Jersey City, NY-NJ-PA",New York,NY,2017,DOMESTICMIG,-203430


### Population -  Pivot the data so that each row has all the population data for a single region and year

In [38]:
# Pivot to make the rows into columns - now every row has the primary key of city, state, and year
df_census_pivot = df_census_melt.pivot(index=['NAME','city','state','year'], columns='census_variable', values='census_value').reset_index()

# Drop NAME
df_census_pivot = df_census_pivot.drop(columns='NAME')

# Make columns census specific
id_cols = ['city','state','year']
value_cols = [i for i in df_census_pivot.columns if i not in id_cols]
new_cols = [i.lower()+'_census' for i in value_cols]

col_dict = {}
count = 0
for i in range(len(value_cols)):
    col_dict[value_cols[i]] = new_cols[count]
    count += 1
    
df_census_pivot = df_census_pivot.rename(col_dict, axis=1)

df_census_pivot.head()

census_variable,city,state,year,births_census,censuspop_census,deaths_census,domesticmig_census,estimatesbase_census,internationalmig_census,naturalinc_census,netmig_census,npopchg_census,popestimate_census,residual_census
0,Abilene,TX,2010,541.0,165252.0,410.0,123.0,165252.0,89.0,131.0,212.0,338.0,165590.0,-5.0
1,Abilene,TX,2011,2292.0,NaN,1501.0,68.0,NaN,206.0,791.0,274.0,1061.0,166651.0,-4.0
2,Abilene,TX,2012,2359.0,NaN,1590.0,-456.0,NaN,529.0,769.0,73.0,832.0,167483.0,-10.0
3,Abilene,TX,2013,2391.0,NaN,1691.0,-1064.0,NaN,373.0,700.0,-691.0,32.0,167515.0,23.0
4,Abilene,TX,2014,2385.0,NaN,1597.0,-308.0,NaN,433.0,788.0,125.0,893.0,168408.0,-20.0


### Top 10 MSAs by population in 2020

In [39]:
df_census_melt.query("census_variable == 'POPESTIMATE' & year == 2020").sort_values('census_value', ascending = False).head(10)[['city','state','census_variable','census_value']]

,city,state,census_variable,census_value
4857,New York,NY,POPESTIMATE,19124359
4818,Los Angeles,CA,POPESTIMATE,13109903
4676,Chicago,IL,POPESTIMATE,9406638
4694,Dallas,TX,POPESTIMATE,7694138
4767,Houston,TX,POPESTIMATE,7154478
4973,Washington,DC,POPESTIMATE,6324629
4833,Miami,FL,POPESTIMATE,6173008
4877,Philadelphia,PA,POPESTIMATE,6107906
4625,Atlanta,GA,POPESTIMATE,6087762
4878,Phoenix,AZ,POPESTIMATE,5059909


# Housing ZHVI Data
Zillow Housing Value Index

### Population -  Read in data, and get isolated region and state data

In [40]:
df_zhvi = pd.read_csv('Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

# Filter for only MSAs
df_zhvi = df_zhvi.query("RegionType == 'Msa'").reset_index(drop=True)

# Change problematic apostrophes in RegionName
df_zhvi.RegionName = [df_zhvi.RegionName.loc[i].replace("'", '^') for i in range(len(df_zhvi))]

# Drop unnecessary columns & rename StateName
df_zhvi = df_zhvi.drop(columns=['RegionID','SizeRank','RegionType'])
df_zhvi = df_zhvi.rename({'StateName':'state'}, axis=1)

# Get the primary city
df_zhvi.insert(loc=1, column='city', value=[i.split('-')[0] for i in df_zhvi.RegionName])
df_zhvi.city = [df_zhvi.city.loc[i].split(',')[0] for i in range(len(df_zhvi))]
df_zhvi

,RegionName,city,state,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,...,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,"New York, NY",New York,NY,225438.0,226785.0,227997.0,230383.0,232604.0,234969.0,237454.0,240026.0,242606.0,245159.0,247613.0,249910.0,252122.0,254555.0,256934.0,259418.0,261773.0,264319.0,266796.0,269475.0,272188.0,275180.0,277937.0,280597.0,283283.0,285900.0,288448.0,290906.0,293506.0,296454.0,299629.0,303552.0,307660.0,311762.0,315400.0,319080.0,322784.0,326589.0,330669.0,334467.0,338342.0,342056.0,345984.0,349766.0,353211.0,356423.0,359423.0,...,448732.0,450575.0,452292.0,453957.0,455660.0,457787.0,460405.0,463361.0,466397.0,468780.0,470440.0,471151.0,471856.0,473039.0,474740.0,476592.0,478161.0,479178.0,480058.0,480959.0,481965.0,482654.0,482852.0,482820.0,482546.0,482079.0,481540.0,481244.0,481784.0,482818.0,484301.0,485725.0,487222.0,488035.0,488594.0,489815.0,492415.0,496938.0,502448.0,508501.0,514350.0,519193.0,523962.0,528241.0,532983.0,538439.0,545638.0,553551.0,561136.0,567059.0
1,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,CA,231209.0,232014.0,233247.0,235592.0,238105.0,240479.0,242910.0,245461.0,247891.0,250113.0,252170.0,254337.0,256001.0,257805.0,259727.0,262406.0,265034.0,267457.0,269935.0,272519.0,275198.0,277854.0,280168.0,282603.0,284611.0,286977.0,289795.0,293379.0,297456.0,301615.0,306850.0,312302.0,317852.0,322809.0,327730.0,332593.0,336688.0,340469.0,344653.0,349330.0,355159.0,359770.0,365789.0,372077.0,379534.0,385684.0,391663.0,...,617400.0,619601.0,624373.0,630304.0,635487.0,640399.0,645573.0,650599.0,655148.0,657699.0,660565.0,660997.0,661875.0,662325.0,665582.0,667436.0,666925.0,665764.0,663614.0,662272.0,661841.0,662809.0,663827.0,662891.0,663616.0,666330.0,670631.0,672643.0,674454.0,677430.0,683527.0,690489.0,695613.0,693703.0,690803.0,692544.0,701716.0,712332.0,722297.0,729787.0,736123.0,741232.0,748830.0,756443.0,767598.0,783132.0,803315.0,824025.0,837548.0,846526.0
2,"Chicago, IL",Chicago,IL,170602.0,171004.0,171525.0,172568.0,173673.0,174768.0,175979.0,177338.0,178743.0,180125.0,181380.0,182607.0,183756.0,184773.0,185926.0,187060.0,188457.0,189749.0,191055.0,192376.0,193710.0,195136.0,196320.0,197391.0,198304.0,199368.0,200651.0,201729.0,202936.0,204042.0,205524.0,207038.0,208502.0,209848.0,210811.0,211975.0,213097.0,214459.0,215686.0,217089.0,218296.0,219399.0,220420.0,221734.0,223214.0,224614.0,225854.0,...,227615.0,228268.0,229173.0,230317.0,231756.0,233362.0,234721.0,235523.0,236008.0,236462.0,236897.0,237309.0,237609.0,238258.0,238762.0,239539.0,240159.0,240695.0,241438.0,241904.0,242413.0,242472.0,242399.0,242359.0,242290.0,242286.0,242178.0,242186.0,242399.0,242882.0,243491.0,244022.0,244150.0,243982.0,243952.0,244966.0,246635.0,249528.0,252392.0,255201.0,257432.0,258905.0,261216.0,263319.0,266367.0,269105.0,272894.0,277086.0,281183.0,284285.0
3,"Dallas-Fort Worth, TX",Dallas,TX,130189.0,130293.0,130379.0,130591.0,130813.0,130998.0,131197.0,131563.0,132018.0,132548.0,133032.0,133525.0,133737.0,133416.0,133

### ZHVI -   Melt the dataframe and get year data so that every row has city, state, year and a census variable

In [41]:
# Melt the dataframe to get one observation per row
id_cols = ['RegionName','city','state']
value_cols = [i for i in df_zhvi.columns if i not in id_cols]
df_zhvi_melt = df_zhvi.melt(id_vars=id_cols, value_vars = value_cols, var_name='zhvi_variable', value_name='zhvi_value')

# Get the year
df_zhvi_melt.insert(loc=3, column='year', value = [int(re.findall("[0-9][0-9][0-9][0-9]", i)[0]) for i in df_zhvi_melt.zhvi_variable])

df_zhvi_melt

,RegionName,city,state,year,zhvi_variable,zhvi_value
0,"New York, NY",New York,NY,2000,2000-01-31,225438.0
1,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,CA,2000,2000-01-31,231209.0
2,"Chicago, IL",Chicago,IL,2000,2000-01-31,170602.0
3,"Dallas-Fort Worth, TX",Dallas,TX,2000,2000-01-31,130189.0
4,"Philadelphia, PA",Philadelphia,PA,2000,2000-01-31,130108.0
...,...,...,...,...,...,...
236983,"Lamesa, TX",Lamesa,TX,2021,2021-09-30,NaN
236984,"Craig, CO",Craig,CO,2021,2021-09-30,NaN
236985,"Pecos, TX",Pecos,TX,2021,2021-09-30,155713.0
236986,"Vernon, TX",Vernon,TX,2021,2021-09-30,NaN


### Get Median ZHVI for each region and year

In [52]:
df_zhvi_meds = df_zhvi_melt.groupby(['city','state','year']).median().reset_index()
df_zhvi_meds

,city,state,year,zhvi_value
0,Aberdeen,SD,2000,NaN
1,Aberdeen,SD,2001,NaN
2,Aberdeen,SD,2002,NaN
3,Aberdeen,SD,2003,NaN
4,Aberdeen,SD,2004,NaN
...,...,...,...,...
19971,Zapata,TX,2017,87775.5
19972,Zapata,TX,2018,89955.0
19973,Zapata,TX,2019,94402.0
19974,Zapata,TX,2020,96690.5


In [42]:
# # Get median zhvi for every year in a region
# regions, states, cities, years, meds = [],[],[],[],[]

# print("Length:", len(set(df_zhvi_melt.RegionName)), end=' ')
# count = 0
# for region in set(df_zhvi_melt.RegionName):
#     count += 1
#     if count % 10 == 0:
#         print(count, end=', ')
#     df_temp = df_zhvi_melt.query(F"RegionName == '{region}'").reset_index(drop=True)
#     city = df_temp.city.loc[0]
#     state = df_temp.state.loc[0]
        
#     for year in set(df_temp.year):
#         med_zhvi = df_temp.query(F"year == {year}").zhvi_value.median()
        
#         regions.append(region)
#         states.append(state)
#         cities.append(city) 
#         years.append(year)
#         meds.append(med_zhvi)

# df_zhvi_meds = pd.DataFrame({
#     'region':regions,
#     'city':cities,
#     'state':states,
#     'year':years,
#     'med_zhvi':meds
# })

# df_zhvi_meds = df_zhvi_meds.drop(columns='region')
# df_zhvi_meds

Length: 908 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 

,city,state,year,med_zhvi
0,Seneca Falls,NY,2000,66434.5
1,Seneca Falls,NY,2001,71229.5
2,Seneca Falls,NY,2002,76359.0
3,Seneca Falls,NY,2003,81548.5
4,Seneca Falls,NY,2004,92045.0
...,...,...,...,...
19971,Des Moines,IA,2017,186841.5
19972,Des Moines,IA,2018,196728.5
19973,Des Moines,IA,2019,203314.5
19974,Des Moines,IA,2020,208956.0


# Housing ZORI Data
Zillow Observed Rent Index

In [43]:
df_zori = pd.read_csv('Metro_ZORI_AllHomesPlusMultifamily_SSA.csv')

# Drop United State from the df
df_zori = df_zori.query("RegionName != 'United States'").reset_index(drop=True)

# Change problematic apostrophes in RegionName
df_zori.RegionName = [df_zori.RegionName.loc[i].replace("'", '^') for i in range(len(df_zori))]

# Drop unnecessary columns & rename StateName
df_zori = df_zori.drop(columns=['RegionID','SizeRank'])
df_zori = df_zori.rename({'StateName':'primary_state'}, axis=1)

# Get the primary (first listed) state of each area
df_zori.insert(loc=1, column='state', value=[i.split(', ')[1].split('-')[0] for i in df_zori.RegionName])

# Get the primary city
df_zori.insert(loc=1, column='city', value=[i.split('-')[0] for i in df_zori.RegionName])
df_zori.city = [df_zori.city.loc[i].split(',')[0] for i in range(len(df_zori))]

df_zori

,RegionName,city,state,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09
0,"New York, NY",New York,NY,2294.0,2304,2315.0,2325,2335,2345,2355,2365,2376,2386,2396,2405,2415,2424,2434,2443,2451,2460,2469,2476,2483,2491,2496,2502,2507,2512,2516,2521,2524,2528,2532,2535,2539,2542,2546,2550,2553,2557,2561,2565,2568,2571,2574,2577,2579,2582,2584,2587,2589,2593,2596,2599,2603,2607,2611,2616,2621,2626,2632,2638,2643,2649,2654,2659,2664,2669,2674,2677,2681,2684,2682,2679,2676,2663,2649,2636,2618,2599,2580,2563,2545,2528,2529,2530,2530,2540,2550,2559,2571.0,2583,2594,2609,2623.0
1,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,CA,1702.0,1712,1723.0,1733,1743,1753,1764,1774,1784,1794,1805,1815,1825,1835,1845,1855,1865,1875,1885,1896,1906,1917,1927,1938,1948,1958,1968,1978,1987,1996,2005,2014,2023,2032,2041,2050,2059,2068,2076,2085,2093,2102,2110,2118,2125,2133,2140,2148,2155,2162,2170,2177,2185,2193,2200,2208,2216,2223,2231,2238,2245,2252,2259,2266,2272,2278,2285,2290,2295,2300,2301,2303,2305,2303,2302,2301,2299,2297,2295,2296,2296,2296,2307,2318,2328,2343,2358,2373,2389.0,2405,2421,2438,2455.0
2,"Chicago, IL",Chicago,IL,1418.0,1422,1426.0,1430,1433,1437,1441,1444,1448,1451,1455,1458,1462,1465,1469,1473,1476,1480,1484,1489,1493,1498,1503,1508,1512,1516,1520,1524,1526,1529,1532,1535,1538,1540,1543,1546,1548,1551,1553,1556,1558,1561,1564,1566,1569,1572,1574,1577,1579,1582,1584,1587,1589,1592,1595,1598,1601,1604,1607,1611,1614,1618,1621,1625,1629,1633,1636,1639,1643,1646,1647,1648,1649,1647,1645,1643,1640,1637,1633,1631,1628,1626,1629,1633,1637,1643,1650,1656,1663.0,1670,1677,1685,1693.0
3,"Dallas-Fort Worth, TX",Dallas,TX,1185.0,1189,1193.0,1196,1200,1204,1208,1212,1216,1220,1224,1228,1232,1237,1242,1247,1252,1256,1261,1266,1271,1276,1281,1286,1291,1297,1302,1308,1313,1319,1324,1329,1335,1340,1344,1349,1354,1358,1363,1368,1372,1376,1381,1385,1389,1394,1397,1401,1405,1409,1412,1416,1420,1424,1428,1432,1437,1441,1446,1450,1455,1459,1464,1468,1472,1476,1481,1484,1487,1491,1493,1495,1497,1498,1500,1501,1503,1506,1508,1513,1518,1523,1536,1549,1562,1579,1595,1612,1629.0,1646,1663,1681,1699.0
4,"Philadelphia, PA",Philadelphia,PA,1301.0,1304,1307.0,1309,1312,1315,1317,1320,1323,1325,1328,1331,1335,1337,1340,1343,1346,1349,1353,1356,1360,1364,1367,1370,1374,1377,1380,1383,1386,1389,1392,1395,1398,1400,1403,1405,1408,1410,1413,1415,1418,1421,1424,1427,1430,1432,1436,1439,1442,1446,1449,1453,1457,1461,1465,1469,1473,1477,1481,1485,1489,1493,1497,1501,1505,1509,1513,1516,1520,1523,1526,1528,1530,1532,1534,1536,1538,1540,1542,1545,1548,1551,1558,1565,1572,1580,1589,1598,1606.0,1615,1624,1634,1643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,"Durham, NC",Durham,NC,1096.0,1099,1103.0,1107,1111,1115,1119,1123,1127,1131,1134,1137,1140,1143,1146,1149,1152,1156,1159,1163,1167,1171,1175,1179,1183,1187,1191,1195,1200,1204,1209,1213,1216,1220,1223,1227,1230,1233,1235,1238,1240,1242,1244,1246,1248,1251,1254,1256,1259,1262,1266,1269,1273,12

### ZORI -   Melt the dataframe and get year data so that every row has city, state, year and a census variable

In [44]:
# Melt the dataframe to get one observation per row
id_cols = ['RegionName','city','state']
value_cols = [i for i in df_zori.columns if i not in id_cols]
df_zori_melt = df_zori.melt(id_vars=id_cols, value_vars = value_cols, var_name='zori_variable', value_name='zori_value')

# Get the year
df_zori_melt.insert(loc=3, column='year', value = [int(re.findall("[0-9][0-9][0-9][0-9]", i)[0]) for i in df_zori_melt.zori_variable])

df_zori_melt

,RegionName,city,state,year,zori_variable,zori_value
0,"New York, NY",New York,NY,2014,2014-01,2294.0
1,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,CA,2014,2014-01,1702.0
2,"Chicago, IL",Chicago,IL,2014,2014-01,1418.0
3,"Dallas-Fort Worth, TX",Dallas,TX,2014,2014-01,1185.0
4,"Philadelphia, PA",Philadelphia,PA,2014,2014-01,1301.0
...,...,...,...,...,...,...
9667,"Durham, NC",Durham,NC,2021,2021-09,1499.0
9668,"Port St. Lucie, FL",Port St. Lucie,FL,2021,2021-09,1937.0
9669,"Fort Collins, CO",Fort Collins,CO,2021,2021-09,1668.0
9670,"Boulder, CO",Boulder,CO,2021,2021-09,2116.0


### Get Median ZORI for each region and year

In [53]:
df_zori_meds = df_zori_melt.groupby(['city','state','year']).median().reset_index()
df_zori_meds

,city,state,year,zori_value
0,Akron,OH,2014,799.5
1,Akron,OH,2015,836.0
2,Akron,OH,2016,854.0
3,Akron,OH,2017,870.5
4,Akron,OH,2018,889.0
...,...,...,...,...
827,Youngstown,OH,2017,650.5
828,Youngstown,OH,2018,670.5
829,Youngstown,OH,2019,687.5
830,Youngstown,OH,2020,721.5


In [45]:
# # Get median zori for every year in a region
# regions, states, cities, years, meds = [],[],[],[],[]

# print("Length:", len(set(df_zori_melt.RegionName)), end=' ')
# count = 0
# for region in set(df_zori_melt.RegionName):
#     count += 1
#     if count % 10 == 0:
#         print(count, end=', ')
#     df_temp = df_zori_melt.query(F"RegionName == '{region}'").reset_index(drop=True)
#     city = df_temp.city.loc[0]
#     state = df_temp.state.loc[0]
        
#     for year in set(df_temp.year):
#         med_zori = df_temp.query(F"year == {year}").zori_value.median()
        
#         regions.append(region)
#         states.append(state)
#         cities.append(city) 
#         years.append(year)
#         meds.append(med_zori)

# df_zori_meds = pd.DataFrame({
#     'region':regions,
#     'city':cities,
#     'state':states,
#     'year':years,
#     'med_zori':meds
# })
# df_zori_meds = df_zori_meds.drop(columns='region')
# df_zori_meds

Length: 104 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 

,city,state,year,med_zori
0,Daytona Beach,FL,2016,1138.0
1,Daytona Beach,FL,2017,1199.5
2,Daytona Beach,FL,2018,1276.5
3,Daytona Beach,FL,2019,1342.5
4,Daytona Beach,FL,2020,1402.5
...,...,...,...,...
827,Des Moines,IA,2019,1070.5
828,Des Moines,IA,2020,1090.5
829,Des Moines,IA,2021,1128.0
830,Des Moines,IA,2014,992.5


# Join Datasets

In [54]:
df_zillow = pd.merge(left = df_zhvi_meds, right=df_zori_meds, how='outer', on=['city','state','year'])
df_zillow

,city,state,year,zhvi_value,zori_value
0,Aberdeen,SD,2000,NaN,NaN
1,Aberdeen,SD,2001,NaN,NaN
2,Aberdeen,SD,2002,NaN,NaN
3,Aberdeen,SD,2003,NaN,NaN
4,Aberdeen,SD,2004,NaN,NaN
...,...,...,...,...,...
19971,Zapata,TX,2017,87775.5,NaN
19972,Zapata,TX,2018,89955.0,NaN
19973,Zapata,TX,2019,94402.0,NaN
19974,Zapata,TX,2020,96690.5,NaN


In [55]:
df_master = pd.merge(left= df_census_pivot, right = df_zillow, how='left', on=['city','state','year'])
df_master

,city,state,year,births_census,censuspop_census,deaths_census,domesticmig_census,estimatesbase_census,internationalmig_census,naturalinc_census,netmig_census,npopchg_census,popestimate_census,residual_census,zhvi_value,zori_value
0,Abilene,TX,2010,541.0,165252.0,410.0,123.0,165252.0,89.0,131.0,212.0,338.0,165590.0,-5.0,101849.5,NaN
1,Abilene,TX,2011,2292.0,NaN,1501.0,68.0,NaN,206.0,791.0,274.0,1061.0,166651.0,-4.0,98293.5,NaN
2,Abilene,TX,2012,2359.0,NaN,1590.0,-456.0,NaN,529.0,769.0,73.0,832.0,167483.0,-10.0,100745.0,NaN
3,Abilene,TX,2013,2391.0,NaN,1691.0,-1064.0,NaN,373.0,700.0,-691.0,32.0,167515.0,23.0,101952.5,NaN
4,Abilene,TX,2014,2385.0,NaN,1597.0,-308.0,NaN,433.0,788.0,125.0,893.0,168408.0,-20.0,108733.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4219,Yuma,AZ,2016,3007.0,NaN,1511.0,-441.0,NaN,932.0,1496.0,491.0,1975.0,207162.0,-12.0,139248.0,NaN
4220,Yuma,AZ,2017,3006.0,NaN,1457.0,-40.0,NaN,581.0,1549.0,541.0,2082.0,209244.0,-8.0,147331.0,NaN
4221,Yuma,AZ,2018,2916.0,NaN,1594.0,374.0,NaN,314.0,1322.0,688.0,2001.0,211245.0,-9.0,155471.5,NaN
4222,Yuma,AZ,2019,3104.0,NaN,1659.0,777.0,NaN,720.0,1445.0,1497.0,2937.0,214182.0,-5.0,161821.0,NaN


In [56]:
df_master.query("city == 'New York'")

,city,state,year,births_census,censuspop_census,deaths_census,domesticmig_census,estimatesbase_census,internationalmig_census,naturalinc_census,netmig_census,npopchg_census,popestimate_census,residual_census,zhvi_value,zori_value
2739,New York,NY,2010,61878.0,18897109.0,31931.0,-24138.0,18896285.0,21868.0,29947.0,-2270.0,27152.0,18923437.0,-525.0,404435.5,NaN
2740,New York,NY,2011,247572.0,NaN,135400.0,-86161.0,NaN,103617.0,112172.0,17456.0,129687.0,19053124.0,59.0,390899.0,NaN
2741,New York,NY,2012,243023.0,NaN,131949.0,-111777.0,NaN,98508.0,111074.0,-13269.0,97329.0,19150453.0,-476.0,376842.0,NaN
2742,New York,NY,2013,242529.0,NaN,138295.0,-115585.0,NaN,89001.0,104234.0,-26584.0,77004.0,19227457.0,-646.0,385507.0,NaN
2743,New York,NY,2014,240755.0,NaN,134684.0,-146459.0,NaN,95069.0,106071.0,-51390.0,54681.0,19282138.0,0.0,405311.0,2350.0
2744,New York,NY,2015,242697.0,NaN,138029.0,-169071.0,NaN,104228.0,104668.0,-64843.0,40101.0,19322239.0,276.0,414813.5,2464.5
2745,New York,NY,2016,241453.0,NaN,136119.0,-197835.0,NaN,106460.0,105334.0,-91375.0,14217.0,19336456.0,258.0,426134.5,2530.0
2746,New York,NY,2017,236744.0,NaN,140309.0,-203430.0,NaN,95474.0,96435.0,-107956.0,-11341.0,19325115.0,180.0,445638.5,2572.5
2747,New York,NY,2018,235154.0,NaN,140953.0,-198717.0,NaN,68752.0,94201.0,-129965.0,-35695.0,19289420.0,69.0,470795.5,2609.0
2748,New York,NY,2019,229366.0,NaN,148762.0,-194533.0,NaN,57282.0,80604.0,-137251.0,-56926.0,19232494.0,-279.0,481874.5,2671.5


In [98]:
df2 = df_master.query("year == 2020")[['city','state','popestimate_census']].sort_values('popestimate_census', ascending=False).reset_index(drop=True)

In [112]:
df2 = df2.loc[5:].reset_index(drop=True)
df2

,city,state,popestimate_census
0,Washington,DC,6324629.0
1,Miami,FL,6173008.0
2,Philadelphia,PA,6107906.0
3,Atlanta,GA,6087762.0
4,Phoenix,AZ,5059909.0
...,...,...,...
374,Danville,IL,74855.0
375,Lewiston,ID,63575.0
376,Walla Walla,WA,61292.0
377,Enid,OK,60869.0


In [121]:


quantile_ranges

[[0, 94], [94, 188], [188, 282], [282, 376]]

In [123]:
random.seed(100)

quantile_ranges = []
counter = int(len(df2)/4)

num = 0
for i in range(4):
    qrange = [num]
    qrange.append(counter*(i+1))
    num=counter*(i+1)
    quantile_ranges.append(qrange)

qs = []
print(q)

msas = pd.DataFrame()
for qrange in quantile_ranges:
    count = 0
    while count < 5:
        q = random.randint(qrange[0],qrange[1])
        
        if q in qs:
            continue
        
        qs.append(q)
        
        temp = pd.DataFrame(df2.loc[q]).transpose()
        msas = pd.concat([msas, temp])
        
        count += 1
        
msas.sort_values('popestimate_census', ascending=False).reset_index(drop=True)

368


,city,state,popestimate_census
0,Detroit,MI,4304136.0
1,Cincinnati,OH,2232907.0
2,Charleston,SC,819705.0
3,Ogden,UT,691359.0
4,Wichita,KS,643768.0
5,Myrtle Beach,SC,514488.0
6,Trenton,NJ,367239.0
7,Lincoln,NE,337836.0
8,Rockford,IL,334072.0
9,Kennewick,WA,303501.0


In [68]:
df2.quantile(q=q, axis=0, numeric_only=True, interpolation='linear')

Docstring:
randint(low, high=None, size=None, dtype=int)

Return random integers from `low` (inclusive) to `high` (exclusive).

Return random integers from the "discrete uniform" distribution of
the specified dtype in the "half-open" interval [`low`, `high`). If
`high` is None (the default), then results are from [0, `low`).

.. note::
    New code should use the ``integers`` method of a ``default_rng()``
    instance instead; please see the :ref:`random-quick-start`.

Parameters
----------
low : int or array-like of ints
    Lowest (signed) integers to be drawn from the distribution (unless
    ``high=None``, in which case this parameter is one above the
    *highest* such integer).
high : int or array-like of ints, optional
    If provided, one above the largest (signed) integer to be drawn
    from the distribution (see above for behavior if ``high=None``).
    If array-like, must contain integer values
size : int or tuple of ints, optional
    Output shape.  If the given shape is, 